In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.keras as keras

import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

import datetime
import torch
import os
from PIL import Image
import cv2

In [ ]:
# Define transformations for the input data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and test datasets
train_generator = train_datagen.flow_from_directory(
    'FER2013/train',
    target_size=(48, 48),
    batch_size=128,
    color_mode='grayscale',
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    'FER2013/test',
    target_size=(48, 48),
    batch_size=128,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 28386 images belonging to 8 classes.
Found 7099 images belonging to 8 classes.


In [ ]:
@tf.keras.utils.register_keras_serializable()
class EmotionCNN(Model):
    def __init__(self, trainable=True, dtype=tf.float32, **kwargs):
        super(EmotionCNN, self).__init__(trainable=trainable, dtype=dtype, **kwargs)
        self.conv1 = layers.Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=(48, 48, 1))
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(64, kernel_size=(3, 3), padding='same')
        self.bn2 = layers.BatchNormalization()
        self.conv3 = layers.Conv2D(128, kernel_size=(3, 3), padding='same')
        self.bn3 = layers.BatchNormalization()
        self.pool = layers.MaxPooling2D(pool_size=(2, 2))
        self.dropout = layers.Dropout(0.5)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(256)
        self.fc2 = layers.Dense(8)  # 8 classes for emotion

    def call(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = tf.nn.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = tf.nn.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = tf.nn.relu(x)
        x = self.pool(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = tf.nn.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return tf.nn.softmax(x)

    def get_config(self):
        config = super(EmotionCNN, self).get_config()
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
model = EmotionCNN()

/Users/shantanufuke/miniconda3/envs/pytorch/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-27 22:18:34.714768: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-05-27 22:18:34.714796: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-27 22:18:34.714801: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-27 22:18:34.714821: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-27 22:18:34.714835: I tensorflow/core/common_runtime/pluggable_device/pluggable

In [ ]:
# Call the model on a sample input to build it
sample_input = tf.random.normal([1, 48, 48, 1])
_ = model(sample_input)

In [ ]:
model.summary()

Model: "emotion_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ ?                      │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ ?                      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ ?                      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ ?                      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ ?                      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,275,528 (4.87 MB)

 Trainable params: 1,275,080 (4.86 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
model = EmotionCNN()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Define ModelCheckpoint and EarlyStopping callbacks
model_checkpoint_callback = ModelCheckpoint(filepath='emotion_cnn_checkpoint.keras',
                                            monitor='val_accuracy', save_best_only=True, mode ='max', verbose = 1)
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, mode = 'max')

model.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
        callbacks = [model_checkpoint_callback, early_stopping_callback]
    )

Epoch 1/50


2024-05-26 12:44:51.964968: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Users/shantanufuke/miniconda3/envs/pytorch/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.3781 - loss: 1.5869
Epoch 1: val_accuracy improved from -inf to 0.55966, saving model to emotion_cnn_checkpoint.keras
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 126ms/step - accuracy: 0.3783 - loss: 1.5864 - val_accuracy: 0.5597 - val_loss: 1.2911
Epoch 2/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.5604 - loss: 1.2590
Epoch 2: val_accuracy improved from 0.55966 to 0.61051, saving model to emotion_cnn_checkpoint.keras
222/222 ━━━━━━━━━━━━━━━━━━━━ 28s 123ms/step - accuracy: 0.5605 - loss: 1.2588 - val_accuracy: 0.6105 - val_loss: 1.0905
Epoch 3/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.6101 - loss: 1.1003
Epoch 3: val_accuracy improved from 0.61051 to 0.64896, saving model to emotion_cnn_checkpoint.keras
222/222 ━━━━━━━━━━━━━━━━━━━━ 28s 124ms/step - accuracy: 0.6102 - loss: 1.1002 - val_accuracy: 0.6490 - val_loss: 0.9793
Epoch 4/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.6553 - loss: 0.9777

In [ ]:
def run_emotion_recognition(model_path='emotion_cnn_checkpoint.keras', emotion_labels=None):
    # Load the trained emotion recognition model
    model = load_model(model_path)

    if emotion_labels is None:
        # Define emotion labels if not provided
        emotion_labels = ['Angry', 'Contempt', 'Disgust', 'Fear','Happy','Neutral','Sad','Suprice']

    # Load the face detection cascade classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Start video capture
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Convert frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Process each detected face
        for (x, y, w, h) in faces:
            # Draw bounding box around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Crop the face region for emotion recognition
            face_roi = gray_frame[y:y+h, x:x+w]
            resized_face = cv2.resize(face_roi, (48, 48))
            normalized_face = resized_face / 255.0
            input_face = np.expand_dims(normalized_face, axis=-1)
            input_face = np.expand_dims(input_face, axis=0)

            # Predict emotion
            prediction = model.predict(input_face)
            predicted_class = np.argmax(prediction)
            emotion = emotion_labels[predicted_class]

            # Overlay emotion label on frame
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, emotion, (x, y - 10), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the frame
        cv2.imshow('Emotion Recognition', frame)

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
run_emotion_recognition()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step


2024-05-27 22:18:42.611184: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━